# Reservoir sampling

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
small_database = '/content/drive/MyDrive/data_sets/iris.csv'
big_database = '/content/drive/MyDrive/data_sets/checkouts-by-title.csv'

## Solución literal del pseudo código (lenta)

In [ ]:
tamaño_muestra = 20

#--preparamos el CSV para ser leído a trozos
reader = pd.read_csv(small_database, iterator = True)
#--rellenamos la muestra con las primeras filas del archivo
muestra = reader.get_chunk(tamaño_muestra)
#--inicializamos el iterador de filas
i = tamaño_muestra + 1
try:
    while reader:
        #--leemos la siguiente fila
        fila = reader.get_chunk(1)
        #--generamos un número aleatorio entre [0, i)
        j = np.random.randint(0,i)
        #--si el número aleatorio es menor que el tamaño de muestra, sustituimos la fila
        if j < tamaño_muestra:
            fila.set_index(pd.Index([j]), inplace = True)
            muestra.update(fila)
        #--incrementamos el iterador de filas
        i += 1
except StopIteration:
    pass
finally:
    del reader


print(muestra.head())


## Solución vectorizada

In [ ]:
%%time
tamaño_muestra = 5000
tamaño_chunks = 500000

#--preparamos el CSV para ser leído a trozos
reader = pd.read_csv(big_database, iterator = True)
#--rellenamos la muestra con las primeras filas del archivo
muestra = reader.get_chunk(tamaño_muestra)
#--inicializamos el iterador de filas
i = tamaño_muestra + 1
try:
    while reader:
        #--obtener el siguiente trozo de datos
        chunk = reader.get_chunk(tamaño_chunks)
        #--definir un vector que contenga el límite para generar números aleatorios
        upper_bound_random = np.arange(i, i + chunk.shape[0])
        #--generamos los números aleatorios
        j = np.random.randint(0, upper_bound_random)
        #--asignamos los números aleatorios como indices
        chunk.set_index(j, inplace = True)
        #--seleccionamos solo los indices que son menores que 'tamaño de muestra'
        chunk = chunk[chunk.index < tamaño_muestra]
        #--eliminamos los indices repetidos, solo mantenemos el último
        chunk = chunk[~chunk.index.duplicated(keep = 'last')]
        #--actulizamos nuestra muestra con las nuevas filas
        muestra.update(chunk)
        #--incrementar i con el número de muestras leído
        i += tamaño_chunks
        # print(i)
except StopIteration:
    pass
finally:
    del reader


print(muestra.head())
